<a href="https://colab.research.google.com/github/simulate111/General/blob/main/Turku_NREL.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [6]:
#@title 📍 Step 1: Select City (No Map) { run: "auto" }

# 1. Selection Menu
selected_city = "Turku" #@param ["Turku", "Copenhagen", "Stockholm", "Oslo"]

# 2. City Coordinate Database
city_db = {
    "Turku": {"lat": 60.5, "lon": 22.3},
    "Copenhagen": {"lat": 55.7, "lon": 12.6},
    "Stockholm": {"lat": 59.3, "lon": 18.1},
    "Oslo": {"lat": 59.9, "lon": 10.7}
}

# 3. Define Global Variables for PVWatts and ERA5 cells
LAT = city_db[selected_city]["lat"]
LON = city_db[selected_city]["lon"]

print("-" * 30)
print(f"✅ City Selected: {selected_city}")
print(f"📍 Latitude:  {LAT}")
print(f"📍 Longitude: {LON}")
print("-" * 30)
print("You can now run Step 2 (ERA5) or Step 4 (PVWatts).")

------------------------------
✅ City Selected: Turku
📍 Latitude:  60.5
📍 Longitude: 22.3
------------------------------
You can now run Step 2 (ERA5) or Step 4 (PVWatts).


In [9]:
#@title ☀️ Step 4: Extract Typical Climatic Data (NREL PVWatts)
import requests
import pandas as pd
from datetime import datetime, timedelta

# 1. Configuration
API_KEY = "F34fZ40i424LgdCiWuwPQ9u2MF3mkWBj1NDFP76O"
SYSTEM_CAPACITY = 4
AZIMUTH = 180
TILT = 35

# 2. Build the API Request
url = "https://developer.nrel.gov/api/pvwatts/v8.json"
params = {
    'api_key': API_KEY,
    'lat': LAT,
    'lon': LON,
    'system_capacity': SYSTEM_CAPACITY,
    'azimuth': AZIMUTH,
    'tilt': TILT,
    'array_type': 1,
    'module_type': 0,
    'losses': 14,
    'timeframe': 'hourly'
}

print(f"--- 📡 Fetching Typical Climatic Year for {selected_city} ---")

response = requests.get(url, params=params)

if response.status_code == 200:
    res = response.json()['outputs']

    # 3. Create a helper timeline to generate names and 01:00 format
    # We use a non-leap year (like 2023) as a template for a 8760-hour cycle
    base = datetime(2023, 1, 1, 0, 0)
    time_stamps = [base + timedelta(hours=i) for i in range(8760)]

    # 4. Create the Dataframe
    df_nrel = pd.DataFrame({
        'Month_Name': [ts.strftime('%B') for ts in time_stamps],
        'Hour': [ts.strftime('%H:00') for ts in time_stamps],
        'AC_Output_W': res['ac'],
        'Solar_Irradiance_Wm2': res['poa'],
        'Temp_Ambient_C': res['tamb'],
        'Wind_Speed_ms': res['wspd']
    })

    # 5. Export to CSV
    nrel_csv = f"{selected_city}_Typical_Climatic_Year.csv"
    df_nrel.to_csv(nrel_csv, index=False)

    print(f"✅ SUCCESS: Saved to {nrel_csv}")
    print(f"Formatted with 'Month_Name' and 'HH:00' time format.")
    print("-" * 30)
    print(df_nrel.head(10))
else:
    print(f"❌ Error: {response.status_code}")

--- 📡 Fetching Typical Climatic Year for Turku ---
✅ SUCCESS: Saved to Turku_Typical_Climatic_Year.csv
Formatted with 'Month_Name' and 'HH:00' time format.
------------------------------
  Month_Name   Hour  AC_Output_W  Solar_Irradiance_Wm2  Temp_Ambient_C  \
0    January  00:00          0.0                   0.0             4.6   
1    January  01:00          0.0                   0.0             4.3   
2    January  02:00          0.0                   0.0             3.5   
3    January  03:00          0.0                   0.0             4.0   
4    January  04:00          0.0                   0.0             4.2   
5    January  05:00          0.0                   0.0             4.2   
6    January  06:00          0.0                   0.0             4.2   
7    January  07:00          0.0                   0.0             4.1   
8    January  08:00          0.0                   0.0             4.1   
9    January  09:00          0.0                   0.0             4.2   